# UCSB Isla Vista Arrest Heatmaps Trial 3: Clusters

This project involves creating heatmaps to visualize arrest patterns in Isla Vista, CA, from September to November 2024, integrating GeoPy with Folium to map and display the data for an article in UCSB's student-run newspaper, the *Daily Nexus*.

### Project Description
The goal of this project was to create detailed heatmaps to visualize the spatial distribution and concentration of arrests in Isla Vista, CA, during the early academic months of September to November 2024. The heatmaps were developed to  accompany and enhance an article written in the *Daily Nexus*, UC Santa Barbara's independent, student-run newspaper. The article, "*Substance-related and theft-related arrests rise in Isla Vista over the last two months*", was written by me (Anna Gornyitzki) and Aastha Prakash.

### Trial 3: Clusters
When creating the initial heatmaps for the project, the visuals effectively represented where arrests in Isla Vista occurred, providing readers with a clear geographic overview. To enhance the visualization further, I included arrest counts directly on the map, giving readers more context about the exact number of arrests at each location. Upon completing the counts trial, I found that the counts were visually unappealing as they often overlapped, particularly when zooming out on the map. To address this, I will use Python library MarkerCluster imported from folium to dynamically adjust the counts with zooming.

# Heatmap 1: All Arrests

## 1. Importing Required Libraries

In [173]:
import sqlite3
import pandas as pd
from geopy.geocoders import GoogleV3
from geopy.exc import GeocoderTimedOut
import folium
from folium.plugins import HeatMap, MarkerCluster
import time

This block imports all the necessary libraries to carry out the task:
* `sqlite3` for interacting with the SQLite database
* `pandas` for handling data and manipulating data frames
* `geopy.geocoders.GoogleV3` to geocode addresses into geographic coordinates (latitude and longitude).
- `folium` for generating interactive maps.
- `folium.plugins.HeatMap` for creating heatmaps using folium.
- `folium.plugins.MarkerCluster` is used to manage and display clusters of markers on a map.
- `time` for handling time-based operations

## 2. Loading Data and Preparing Addresses

In [174]:
conn = sqlite3.connect('arrestsData.db')
df = pd.read_sql_query('SELECT Location || ", Isla Vista, CA" as address FROM UCSB_Crime_Fire_Log ucfl WHERE Disposition LIKE "Arrest%"', conn)
addresses = df['address'].tolist()
conn.close()

First, a connection is made to the SQLite database `arrestsData.db` using `sqlite3`. Then, a SQL query is executed to retrieve arrest location data from the `UCSB_Crime_Fire_Log` table. The query concatenates `", Isla Vista, CA"` to the `Location` field to form complete addresses. This data is loaded into a pandas DataFrame, and the `address` column is converted into a list of addresses, which will be used for geocoding.

**The main differentiator in this trial is is letting Python handle the counts and Cluster perform the counting for me.**

## 3. Geocoding the Addresses

In [175]:
coordinates = []
for address in addresses:
    coord = geocode_address(address)
    if coord:
        coordinates.append(coord)

This loop iterates through each address in the `addresses` list and calls the `geocode_address()` function to convert the address into geographic coordinates (latitude and longitude). If the geocoding process is successful and returns valid coordinates (`coord`), they are added to the `coordinates` list. The function `geocode_address` is assumed to handle the geocoding logic (not defined in the provided code).

## 4. Setting the Map Center and Creating the Heatmap

In [176]:
map_center = [34.413120041278376, -119.86458253856023]
heat_map = folium.Map(location=map_center, zoom_start=15)
HeatMap(coordinates).add_to(heat_map)

- The `map_center` is set to the geographic coordinates for a central point in Isla Vista, CA.
- A `folium.Map` object is created, centered on `map_center` with an initial zoom level of 15.
- The `HeatMap` object from `folium.plugins` is created using the `coordinates` list (containing latitude and longitude of arrest locations) and added to the `heat_map` object.
The heatmap will visualize the density of arrests by displaying areas with higher concentrations of points in warmer colors.

## 5. Adding Counts to a Marker Cluster

In [178]:
marker_cluster = MarkerCluster().add_to(heat_map)
for lat, lon in coordinates:
    folium.CircleMarker(
        location=[lat, lon],
        radius=8,
        color="light blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.4,
        popup="Count: 1"  # Set a default count of 1
    ).add_to(marker_cluster)

- **Create and Attach Marker Cluster**:  
   - A `MarkerCluster` is initialized and added to the `heat_map`, enabling automatic grouping of markers that are spatially close. This helps reduce clutter and enhances map readability.

- **Add Circle Markers**:  
   - The loop iterates through the `coordinates` list, which contains latitude (`lat`) and longitude (`lon`) pairs. For each pair:
     - A `CircleMarker` is created to represent the data point on the map.
     - Marker properties:
       - **`location`**: Specifies the marker's geographic position using latitude and longitude.
       - **`radius`**: Sets the size of the marker (8 in this case).
       - **`color`** and **`fill_color`**: Define the marker's border (light blue) and fill (blue).
       - **`fill_opacity`**: Sets the transparency level for the fill (0.4 for semi-transparency).
       - **`popup`**: Displays a small popup with the text "Count: 1" when the marker is clicked, indicating a default count.
     - The `CircleMarker` is added to the `MarkerCluster`.

## 6. Saving and Displaying the Map

The generated heatmap is saved as an HTML file using the `save()` method, allowing it to be viewed in a web browser. The heatmap is then displayed and provides an interactive visualization of arrest concentrations across Isla Vista.

In [179]:
heat_map.save("generalHeatmapCluster.html")
heat_map

# Heatmap 2: Substance-related Arrests

The same process is repeated for substance-related arrests as was for all arrests except the only difference is the SQL query used.

In [180]:
import sqlite3
import pandas as pd
from geopy.geocoders import GoogleV3
from geopy.exc import GeocoderTimedOut
import folium
from folium.plugins import HeatMap, MarkerCluster
import time

conn = sqlite3.connect('arrestsData.db')
df = pd.read_sql_query("SELECT Location || ', Isla Vista, CA' as address, Classification FROM UCSB_Crime_Fire_Log ucfl WHERE Disposition LIKE 'Arrest%' AND (upper(Classification) LIKE '%ALCOHOL%' OR upper(Classification) LIKE '%INTOXICATION%' OR upper(Classification) LIKE '%DRUG%' OR upper(Classification) LIKE '%OPEN CONTAINER%' OR upper(Classification) LIKE '%SUBSTANCE%' OR upper(Classification) LIKE '%INFLUENCE%'OR upper(Classification) LIKE '%POSSESS%' OR upper(Classification) LIKE '%NARCOTICS%')", conn)

A SQL query is executed to retrieve arrest data for substance-related offenses from the `UCSB_Crime_Fire_Log` table. The query filters rows based on the `Disposition` field (to include only "Arrest" entries) and matches keywords related to substance use (e.g., "ALCOHOL," "DRUG," "INTOXICATION"). It also concatenates `", Isla Vista, CA"` to the `Location` field to form complete addresses.

In [181]:
addresses = df['address'].tolist()
conn.close()

coordinates = []
for address in addresses:
    coord = geocode_address(address)
    if coord:
        coordinates.append(coord)

map_center = [34.413120041278376, -119.86458253856023]
s_heat_map = folium.Map(location=map_center, zoom_start=15)
HeatMap(coordinates).add_to(s_heat_map)

marker_cluster = MarkerCluster().add_to(s_heat_map)
for lat, lon in coordinates:
    folium.CircleMarker(
        location=[lat, lon],
        radius=8,
        color="light blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.4,
        popup="Count: 1"  # Set a default count of 1
    ).add_to(marker_cluster)

s_heat_map.save("substanceHeatmapCluster.html")
s_heat_map

# Heatmap 3: Theft and property-related Arrests

The same process is repeated for theft and property-related arrests as was for all arrests except the only difference is the SQL query used.

In [182]:
import sqlite3
import pandas as pd
from geopy.geocoders import GoogleV3
from geopy.exc import GeocoderTimedOut
import folium
from folium.plugins import HeatMap, MarkerCluster
import time

conn = sqlite3.connect('arrestsData.db')
df = pd.read_sql_query("SELECT Location || ', Isla Vista, CA' as address, Classification FROM UCSB_Crime_Fire_Log ucfl WHERE Disposition LIKE 'Arrest%' AND (upper(Classification) LIKE '%THEFT%' OR upper(Classification) LIKE '%LARCENY%' OR upper(Classification) LIKE '%SHOPLIFTING%' OR upper(Classification) LIKE '%VANDALISM%' OR upper(Classification) LIKE '%TRESPASS%' OR upper(Classification) LIKE '%PROPERTY%'OR upper(Classification) LIKE '%BURGLARY%')", conn)

A SQL query is executed to retrieve arrest data for substance-related offenses from the UCSB_Crime_Fire_Log table. The query filters rows based on the Disposition field (to include only "Arrest" entries) and matches keywords related to theft and property crimes (e.g., "THEFT," "SHOPLIFTING," "BURGLARY"). It also concatenates ", Isla Vista, CA" to the Location field to form complete addresses.

In [185]:
addresses = df['address'].tolist()
conn.close()

coordinates = []
for address in addresses:
    coord = geocode_address(address)
    if coord:
        coordinates.append(coord)

map_center = [34.413120041278376, -119.86458253856023]
t_heat_map = folium.Map(location=map_center, zoom_start=15)
HeatMap(coordinates).add_to(t_heat_map)

marker_cluster = MarkerCluster().add_to(t_heat_map)
for lat, lon in coordinates:
    folium.CircleMarker(
        location=[lat, lon],
        radius=8,
        color="light blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.4,
        popup="Count: 1"  # Set a default count of 1
    ).add_to(marker_cluster)

t_heat_map.save("theftHeatmapCluster.html")
t_heat_map

## Trial 3 Conclusion

These are the final versions of the heatmaps I embedded in my *Daily Nexus* article. I’m satisfied with how interactive the maps are and how visually appealing the counts look. The final product not only enhances the article but also provides an engaging and informative experience for readers.